In [ ]:
from langchain_chroma  import Chroma
from nlp_chat_bot.model.stella import Stella
from nlp_chat_bot.rag import RAG
from nlp_chat_bot.model.late_chunking_embedding import LateChunkingEmbedding

In [ ]:
dataset_path = "../data"
model_download_path = "../models"

embedding_function = LateChunkingEmbedding(model_download_path, chunk_size=1000)
vector_store = Chroma(embedding_function=embedding_function)
rag = RAG(dataset_path, vector_store)
print("LENGTH", len(vector_store.get()['documents']))
docs_retrieved = rag.retrieve(state = {"question": "What is the acronym AIA?", "context": []})

print("Num docs:", len(docs_retrieved["context"]))

for i in range(len(docs_retrieved["context"])):
    doc = docs_retrieved["context"][i]
    print("\n\n", "#"*30,"\n")
    print(f"doc {i}: (score: {doc.metadata['score']})")
    print(doc.page_content)
print(docs_retrieved["context"][0].page_content)